# S&P 500 stock - a descriptive analysis with Spark

In [28]:
import numpy as np  
import pandas as pd  
!pip install pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, length

In [29]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("S&P500StockAnalysis") \
    .getOrCreate()


In [30]:
# Define the path to the CSV file
csv_file_path = "/kaggle/input/sandp500/all_stocks_5yr.csv"

# Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
print(f'# rows: {df.count()}')
df.show()

# rows: 619040
+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.0

In [31]:
df.describe().show()

+-------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|            open|             high|              low|            close|           volume|  Name|
+-------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|          619029|           619032|           619032|           619040|           619040|619040|
|   mean|83.0233343145481|83.77831069347307|82.25609641375267|83.04376276476548|4321823.395568945|  null|
| stddev|97.3787690433234|98.20751890446383|96.50742105809076|97.38974800165785| 8693609.51196759|  null|
|    min|            1.62|             1.69|              1.5|             1.59|                0|     A|
|    max|          2044.0|          2067.99|          2035.11|           2049.0|        618237630|   ZTS|
+-------+----------------+-----------------+-----------------+-----------------+-----------------+------+



Quantos registros há na planilha para a ação da Apple (AAPL)?

In [32]:
# Count the occurrences of "AAPL"
count_aapl = df.filter(df["Name"] == "AAPL").count()
print(f"Existem {count_aapl} ocorrências para o código AAPL")

Existem 1259 ocorrências para o código AAPL


#### Quantas empresas distintas têm registros nessa planilha?

In [33]:
# Get distinct or unique values in the "Name" column
numer_of_unique_stocks  = df.select("Name").distinct().count()
print(f"Existem {numer_of_unique_stocks} ações distintas no banco de dados")
 

Existem 505 ações distintas no banco de dados


#### Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

In [34]:
CloseGreatOpen = df.filter(df["Close"] > df["Open"]).count()
FreqCloseGreatOpen = 100*CloseGreatOpen/df.count()
print(f"O preço das ações no fechamento é maior do que na abertura em {FreqCloseGreatOpen:.2f}% dos casos")

O preço das ações no fechamento é maior do que na abertura em 51.53% dos casos


#### Qual o maior valor das ações da Apple (AAPL) na história?

In [35]:
aapl = df.filter(df["Name"] == "AAPL")
biggest_aapl_value = aapl.agg({"high": "max"}).collect()[0][0]
print(f"O maior valor das ações da Apple (AAPL) na história foi de US$ {biggest_aapl_value}")

O maior valor das ações da Apple (AAPL) na história foi de US$ 180.1


####  Qual ação tem a maior volatilidade? 

Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [36]:
greatest_std = df.groupby("Name").agg({"close": "std"}).orderBy(col("stddev(close)").desc()).first()
print(f"A ação com maior volatilidade foi {greatest_std['Name']}, com um desvio padrão no preço de fechamento de US$ {greatest_std['stddev(close)']:.2f}")

A ação com maior volatilidade foi PCLN, com um desvio padrão no preço de fechamento de US$ 320.53


#### Qual o dia com maior volume total de negociação da bolsa?

In [37]:
greatest_volume_data = df.groupby("date").agg({"volume": "sum"}).orderBy(col("sum(volume)").desc()).first()
print(f"O dia com maior volume de negociação na bolsa foi {greatest_volume_data['date']}, com um total de US$ {greatest_volume_data['sum(volume)']:.2f}")

O dia com maior volume de negociação na bolsa foi 2015-08-24, com um total de US$ 4607945196.00


#### Qual a ação mais negociada da bolsa, em volume de transações?

In [38]:
most_traded_stock = df.groupby("Name").agg({"volume": "sum"}).orderBy(col("sum(volume)").desc()).first()
print(f"A ação mais negociada da bolsa, em volume de transações, foi {most_traded_stock['Name']}, com um total de US$ {most_traded_stock['sum(volume)']:.2f}")

A ação mais negociada da bolsa, em volume de transações, foi BAC, com um total de US$ 117884953591.00


#### Quantas ações começam com a letra “A”?


In [39]:
starts_with_A = df.select("Name").distinct().filter(col("Name").startswith("A")).count()
print(f"{starts_with_A} ações começam com a letra A")

59 ações começam com a letra A


#### Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?


In [40]:
HighIgualClose = df.filter(df["high"] == df["close"]).count()
FreqHighIgualClose = 100*HighIgualClose/df.count()
print(f"O preço mais alto do dia da ação também é o preço de fechamento em {FreqHighIgualClose:.2f}% das ocasiões")

O preço mais alto do dia da ação também é o preço de fechamento em 1.20% das ocasiões


#### Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?


In [41]:
aapl_exp = aapl.withColumn('diff_open_close', aapl['open'] - aapl['close'])
max_abs_diff_day = aapl_exp.select("date", "diff_open_close").orderBy(col("diff_open_close").desc()).first()
print(f"A ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta, no dia {max_abs_diff_day['date']}")

A ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta, no dia 2015-08-25


#### Em média, qual o volume diário de transações das ações da AAPL?

In [42]:
avg_volume_aapl = aapl.agg({"volume": "mean"}).collect()[0][0]
print(f"O volume diário médio de transações das ações da AAPL é de US$ {avg_volume_aapl:.2f}")

O volume diário médio de transações das ações da AAPL é de US$ 54047899.74


#### Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

In [43]:
name_df = df.groupby("Name").count()
# Adicione uma coluna para calcular o comprimento do nome
name_df = name_df.withColumn("name_length", length(col("Name")))
groupby_name_length = name_df.groupby("name_length").count().\
                                        orderBy(col("name_length").asc()).show()
groupby_name_length

+-----------+-----+
|name_length|count|
+-----------+-----+
|          1|   10|
|          2|   50|
|          3|  323|
|          4|  117|
|          5|    5|
+-----------+-----+



#### Qual a ação menos negociada da bolsa, em volume de transações?

In [44]:
least_traded_stock = df.groupby("Name").agg({"volume": "sum"}).orderBy(col("sum(volume)").asc()).first()
print(f"A ação menos negociada da bolsa foi {least_traded_stock['Name']}, com um volume total de US$ {least_traded_stock['sum(volume)']:.2f}")

A ação menos negociada da bolsa foi APTV, com um volume total de US$ 92947779.00


In [45]:
# Finaliza a sessão 
spark.stop()